# Introduction

based on : https://github.com/bentrevett/pytorch-sentiment-analysis

one-layer RNN Classifier with IMDB datasets

# 0. Set Environment

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import torchtext
torchtext.disable_torchtext_deprecation_warning()
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint

import subprocess
import os
import sys

import datasets

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Selected device:", device)

Selected device: cuda


In [3]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
model_dir = './models/LSTM_Classifier_model.pth'
pretrained_embedding_dir = './models/Glove_pretrained.pth'

# 1. Data processing

## 1-1. Get Data

In [5]:
train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])

## 1-2. Tokenize

In [6]:
tokenizer = get_tokenizer("basic_english")

In [7]:
def tokenize_example(example, tokenizer, max_length):
    tokens = tokenizer(example["text"])[:max_length]
    length = len(tokens)
    return {"tokens": tokens, "length": length}

In [8]:
max_length = 256

train_data = train_data.map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length}
)
test_data = test_data.map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length}
)

## 1-3. Build Vocab 

In [9]:
min_freq = 5
special_tokens = ["<unk>", "<pad>"]

vocab = build_vocab_from_iterator(train_data['tokens'],
                                  min_freq = min_freq,
                                  specials = special_tokens)

In [10]:
unk_index = vocab["<unk>"]
pad_index = vocab["<pad>"]

vocab.set_default_index(unk_index)

## 1-4. Numericalize Text

In [11]:
def numericalize_example(example, vocab):
    ids = vocab.lookup_indices(example["tokens"])
    return {"ids": ids}

In [12]:
train_data = train_data.map(numericalize_example, fn_kwargs={"vocab": vocab})
test_data = test_data.map(numericalize_example, fn_kwargs={"vocab": vocab})

In [13]:
train_data = train_data.with_format(type="torch", columns=["ids", "label", "length"])
test_data = test_data.with_format(type="torch", columns=["ids", "label", "length"])

## 1-5. Word Embedding

In [14]:
if not os.path.exists(pretrained_embedding_dir):
    vectors = torchtext.vocab.GloVe()
    pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
    torch.save(pretrained_embedding, pretrained_embedding_dir)
else:
    pretrained_embedding = torch.load(pretrained_embedding_dir)

## 1-6. Prepare for Data Loading

In [15]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        return item

In [16]:
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

In [17]:
def custom_collate_fn(batch):
    
    batch_inputs = [sample['ids'] for sample in batch]
    batch_labels = [sample['label'] for sample in batch]
    
    collate_inputs = pad_sequence(batch_inputs, 
                                  padding_value = pad_index, 
                                  batch_first = True)
    collate_labels = torch.tensor(batch_labels)
    
    return collate_inputs, collate_labels

In [18]:
batch_size = 128
pad_index = pad_index
shuffle = True

trainloader = DataLoader(dataset = train_dataset,
                         batch_size = batch_size,
                         collate_fn = custom_collate_fn,
                         shuffle = shuffle)
testloader = DataLoader(dataset = test_dataset,
                         batch_size = batch_size,
                         collate_fn = custom_collate_fn,
                         shuffle = shuffle)

# 2. Define Model

## 2-1. Model Structure

In [19]:
class LSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, bias = True):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.bias = bias

        self.Wi = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        self.Wf = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        self.Wo = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        self.Wh = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        
    def forward(self, input, hidden):
        h, c = hidden
        concat_ih = torch.cat((input, h), 1)

        input_gate = F.sigmoid(self.Wi(concat_ih))
        forget_gate = F.sigmoid(self.Wf(concat_ih))
        output_gate = F.sigmoid(self.Wo(concat_ih))
        cell_gate = F.tanh(self.Wh(concat_ih))

        c = forget_gate * c + input_gate * cell_gate
        h = output_gate * F.tanh(c)

        return h, c

In [20]:
class DeepLSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, output_dim, pad_index, bias = True):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_index)
        self.embedding.requires_grad_(False)
        
        self.Layers = nn.ModuleList([LSTMCell(embedding_dim, hidden_dim, bias=bias) for _ in range(num_layers)])

        self.fc1 = nn.Linear(hidden_dim, embedding_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, inputs):

        batch_size = inputs.size(0)
        seq_length = inputs.size(1)

        x = self.embedding(inputs)

        
        for i in range(len(self.Layers)):

            LSTM = self.Layers[i]
            
            h0 = torch.zeros(batch_size, self.hidden_dim).to(device)
            c0 = torch.zeros(batch_size, self.hidden_dim).to(device)
            hidden = [h0, c0]

            for t in range(seq_length):

                input_t = x[:, t, :]
                h, c = LSTM.forward(input_t, hidden)
                hidden = [h, c]
                x[:, t, :] = self.fc1(h)

        output = self.fc2(h)

        return output

## 2-2. Hyperparameter & functions

In [21]:
vocab_size = len(vocab)
embedding_dim = 300
hidden_dim = 128
num_layers = 3
output_dim = 1
pad_index = pad_index
lr = 5e-4

model = DeepLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, output_dim, pad_index, bias = True)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## 2-3. Weight Initialization

In [22]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, LSTMCell):
        for name, param in m.named_parameters():
            if "bias" in name:
                nn.init.zeros_(param)
            else:
                nn.init.orthogonal_(param)

In [23]:
model.apply(initialize_weights)

DeepLSTM(
  (embedding): Embedding(24897, 300, padding_idx=1)
  (Layers): ModuleList(
    (0-2): 3 x LSTMCell(
      (Wi): Linear(in_features=428, out_features=128, bias=True)
      (Wf): Linear(in_features=428, out_features=128, bias=True)
      (Wo): Linear(in_features=428, out_features=128, bias=True)
      (Wh): Linear(in_features=428, out_features=128, bias=True)
    )
  )
  (fc1): Linear(in_features=128, out_features=300, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [24]:
model.embedding.weight.data = pretrained_embedding

In [25]:
pprint("Model's state_dict:")
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"    Size : {param.size()}")
    print(f"    Value: {param}")

"Model's state_dict:"
Parameter name: embedding.weight
    Size : torch.Size([24897, 300])
    Value: Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2720, -0.0620, -0.1884,  ...,  0.1302, -0.1832,  0.1323],
        ...,
        [ 0.6701, -0.2717,  0.4766,  ...,  0.2786,  0.3312,  0.0230],
        [-0.1503,  0.5624, -0.5622,  ..., -0.4224, -0.6836,  0.0726],
        [ 1.1741, -0.4386,  0.3310,  ...,  0.3193, -0.2292, -0.0887]])
Parameter name: Layers.0.Wi.weight
    Size : torch.Size([128, 428])
    Value: Parameter containing:
tensor([[-0.0691, -0.0469, -0.0069,  ..., -0.0463,  0.0331,  0.0482],
        [-0.0132,  0.0432,  0.0836,  ...,  0.0699, -0.0441, -0.1006],
        [ 0.0343,  0.0028, -0.1117,  ...,  0.0473,  0.0246, -0.0328],
        ...,
        [-0.0366,  0.0080,  0.0791,  ..., -0.0270,  0.0119, -0.0892],
        [-0.0089,  0.0095,  0.0562,  ..., -0.03

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 697,773 trainable parameters


In [27]:
model.to(device)
criterion.to(device)

BCEWithLogitsLoss()

## 2-4. Tensorboard

In [28]:
writer = SummaryWriter()

# 3. Train Model

In [29]:
def train_model(model, criterion, optimizer, trainloader, num_epochs):
    print("-----Training Started------")
    for epoch in range(num_epochs):
        
        model.train()
        
        running_loss = 0.0
        
        for inputs, labels in tqdm(trainloader):
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels.float())
            loss.backward()
            optimizer.step()

            writer.add_scalar('Loss/train', loss.item(), epoch)

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(trainloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss: .4f}")

        torch.save(model.state_dict(), model_dir)
    
    writer.close()
    
    print("-----Training Completed-----")

In [30]:
num_epochs = 16

train_model(model, criterion, optimizer, trainloader, num_epochs)

-----Training Started------


100%|██████████| 196/196 [02:51<00:00,  1.14it/s]


Epoch [1/16], Loss:  0.6808


100%|██████████| 196/196 [02:51<00:00,  1.14it/s]


Epoch [2/16], Loss:  0.6898


...

Epoch [13/16], Loss:  0.6932


100%|██████████| 196/196 [02:50<00:00,  1.15it/s]


Epoch [14/16], Loss:  0.6932


 42%|████▏     | 82/196 [01:12<01:40,  1.13it/s]


KeyboardInterrupt: 

# 4. Test

In [ ]:
def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    TP = 0  # True Positives
    TN = 0  # True Negatives
    FP = 0  # False Positives
    FN = 0  # False Negatives

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            predictions = torch.round(F.sigmoid(outputs))
            total += labels.size(0)

            predictions, labels = predictions.view(-1).cpu(), labels.cpu()
            
            correct += (predictions == labels).sum().item()

            TP += ((predictions == 1) & (labels == 1)).sum().item()
            TN += ((predictions == 0) & (labels == 0)).sum().item()
            FP += ((predictions == 1) & (labels == 0)).sum().item()
            FN += ((predictions == 0) & (labels == 1)).sum().item()

    accuracy = correct / total
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    print(f"Accuracy on test set: {accuracy:.4f}")
    print(f"Precision on test set: {precision:.4f}")
    print(f"Recall on test set: {recall:.4f}")
    print(f"F1 Score on test set: {f1:.4f}")


In [ ]:
model.load_state_dict(torch.load(model_dir))
model.to(device)

SimpleRNN(
  (embedding): Embedding(24897, 300, padding_idx=1)
  (Layer): RNNCell(
    (Wi): Linear(in_features=812, out_features=512, bias=True)
  )
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
test_model(model, testloader)

Accuracy on test set: 0.5126
Precision on test set: 0.5326
Recall on test set: 0.2061
F1 Score on test set: 0.2972


# 5. Inference

In [ ]:
def inference(text, model, tokenizer, vocab):
    tokens = tokenizer(text)
    ids = vocab.lookup_indices(tokens)
    tensor = torch.LongTensor(ids).unsqueeze(dim=0).to(device)

    output = model(tensor)

    prediction = torch.round(F.sigmoid(output))

    return prediction

In [ ]:
model.load_state_dict(torch.load(model_dir))
model.to(device)

SimpleRNN(
  (embedding): Embedding(24897, 300, padding_idx=1)
  (Layer): RNNCell(
    (Wi): Linear(in_features=812, out_features=512, bias=True)
  )
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
text = 'This film is terrible!'

prediction = inference(text, model, tokenizer, vocab)

print(prediction.item())

0.0


In [ ]:
text = 'This film is great!'

prediction = inference(text, model, tokenizer, vocab)

print(prediction.item())

1.0


In [ ]:
text = 'The best film I have ever seen!'

prediction = inference(text, model, tokenizer, vocab)

print(prediction.item())

1.0


In [ ]:
text = "This film is not terrible, it's great!"

prediction = inference(text, model, tokenizer, vocab)

print(prediction.item())

0.0


# Limitation

padding 토큰 연산 제외 / dropout / validation